# Initial Visualisation

## Trying out candle stick visualisation with dummy data

In [1]:
# bring all the libs
from plotly.offline import init_notebook_mode, plot
from plotly.graph_objs import Candlestick as candles

import pandas as pd
# import data set
bitcoinData = pd.read_csv('bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2017-05-31.csv')

In [1]:
init_notebook_mode(connected=True)

d = {
    'Open': pd.Series([2, 3, 7, 5]),
    'Close': pd.Series([4, 5, 6, 0]),
    'High': pd.Series([3, 6, 1, 8]),
    'Low': pd.Series([1, 2, 7, 9])
}

df = pd.DataFrame(d)
print(df)

candleGraph = candles(
    x=df.index,
    open=df.Open,
    high=df.High,
    low=df.Low,
    close=df.Close
)

data = [candleGraph]
# Opens a new window with the candles
plot(data)

ModuleNotFoundError: No module named 'plotly'

## Trying out candle stick visualisation with bitcoin data

In [ ]:
bitcoinData.head()
length = len(bitcoinData)
subData = pd.DataFrame(bitcoinData, index=range(length - 10, length))
# Print some of the data
print(subData)

candleGraph = candles(
    x=subData.index,
    open=subData.Open,
    high=subData.High,
    low=subData.Low,
    close=subData.Close
)

data = [candleGraph]
# Opens a new window with the candles
plot(data)

## Viewing the head and tail of the data

In [ ]:
print(bitcoinData.head(10))
print(bitcoinData.tail(10))

# as we see there are many rows that have a NaN as one of the values

# Data Cleaning

Task #1: Remove the rows that have at least 1 NaN value    
Task #2: Find out how many records have at least 1 NaN value    
Task #3: Find out how many records have no NaN values at all    
Task #4: Repeat tasks #2 and #3 giving values in percentages    

In [7]:
import math

# function that removes rows that have NaN values as elements
def remove_NaN_from_dataSet(dataSet):
    non_nan_index = []
    for row_index in dataSet.index:
#         print(row_index)
        non_nan_index.append(row_index)
        for element in dataSet.loc[row_index]:
#             print('  ', element)
            if math.isnan(element):
#                 dataSet = dataSet.drop(row_index)
#                 print('     breaking')
                non_nan_index.pop()
                break
    return pd.DataFrame(dataSet, index=non_nan_index)

print('Removing the NaN values...')
bitcoin_data_without_nan = remove_NaN_from_dataSet(bitcoinData)

# save the non NaN file
print('Saving the new data set...')
bitcoin_data_without_nan.to_csv('bitcoin-historical-data/bitstampUSD_non_nan.csv')

# find the number of records with NaN
print('Calculating the various percentages...')
original_no_records = len(bitcoinData)
non_nan_no_records = len(bitcoin_data_without_nan)
difference = original_no_records - non_nan_no_records
print('Total original bitcoin records: ', original_no_records)
print('Records with NaN:', difference)
perc_nan_records = (difference/original_no_records) * 100
print('% of NaN records: ', perc_nan_records)
print('% of non-NaN records: ', 100 - perc_nan_records)

Removing the NaN values...
Saving the new data set...
Calculating the various percentages...
Total original bitcoin records:  2841377
Records with NaN: 1190473
% of NaN records:  41.89774887316959
% of non-NaN records:  58.10225112683041


# Financial Trading Simulation

### Simulation of Dual Moving Average

In [20]:
# import clean data
print('Reading data...')
cleanBitcoinData = pd.read_csv('bitcoin-historical-data/bitstampUSD_non_nan.csv')        
print('Data read.')

Reading data...
Data read.


In [31]:
import math

def dual_moving_strategy(btc_dataSet, short_moving_interval, long_moving_interval, total_usd_wealth):
    # data set must be greater in row size than long moving average interval
#     print(len(btc_dataSet.index))
#     print(long_moving_interval)
    if len(btc_dataSet.index) < long_moving_interval:
        print('Error: Data set row size must be greater than long moving average.')
        return
    
    close_btc = btc_dataSet['Close']
    usd_per_btc = btc_dataSet['Weighted_Price']
    short_mavg = close_btc.rolling(short_moving_interval).mean()
    long_mavg = close_btc.rolling(long_moving_interval).mean()

    # total wealth (in $)
    usd_wealth = total_usd_wealth
    # total welath (in btc)
    btc_wealth = 0.0

    # true if short_mavg > long_mavg
    # false if long_mavg <= short_mavg
    prev_status = None
    curr_status = None
    for index in range(len(long_mavg)):
        print(btc_dataSet.loc[index])
        prev_status = curr_status
        curr_status = True if short_mavg[index] > long_mavg[index] else False
    
        # if value is not available or the status does not exist (initial stages)
        if math.isnan(long_mavg[index]) or prev_status is None:
            continue
        # buy BTC / sell USD
        elif curr_status == True and prev_status == False:
            # buy BTC worth 10% of wealth
            usd_to_sell = round(usd_wealth * 0.1, 2) 
            btc_to_buy = round(usd_to_sell / usd_per_btc[index], 2)
            usd_wealth = usd_wealth - usd_to_sell
            btc_wealth = btc_wealth + btc_to_buy
            print('buy BTC / sell USD')
            print('usd_to_sell:', usd_to_sell)
            print('Total USD wealth:', usd_wealth)
        # sell USD / buy BTC
        elif curr_status == False and prev_status == True:
            # sell all BTC
            usd_to_buy = round(btc_wealth * usd_per_btc[index], 2)
            usd_wealth = usd_wealth + usd_to_buy
            btc_wealth = 0
            print('sell USD / buy BTC')
            print('usd_to_buy:', usd_to_buy)
            print('Total USD wealth:', usd_wealth)
        print('\n')
    return (usd_wealth, btc_wealth)

subset_btc_data = pd.DataFrame(cleanBitcoinData, index=range(100))
# dual_moving_strategy(btc_dataSet, short_moving_interval, long_moving_interval, total_usd_wealth)
dual_moving_strategy(subset_btc_data, 2, 4, 100)
    

Unnamed: 0           0.000000e+00
Timestamp            1.325318e+09
Open                 4.390000e+00
High                 4.390000e+00
Low                  4.390000e+00
Close                4.390000e+00
Volume_(BTC)         4.555809e-01
Volume_(Currency)    2.000000e+00
Weighted_Price       4.390000e+00
Name: 0, dtype: float64
Unnamed: 0           4.780000e+02
Timestamp            1.325347e+09
Open                 4.390000e+00
High                 4.390000e+00
Low                  4.390000e+00
Close                4.390000e+00
Volume_(BTC)         4.800000e+01
Volume_(Currency)    2.107200e+02
Weighted_Price       4.390000e+00
Name: 1, dtype: float64
Unnamed: 0           5.470000e+02
Timestamp            1.325351e+09
Open                 4.500000e+00
High                 4.570000e+00
Low                  4.500000e+00
Close                4.570000e+00
Volume_(BTC)         3.786230e+01
Volume_(Currency)    1.713803e+02
Weighted_Price       4.526411e+00
Name: 2, dtype: float64
Unnamed: 0

(102.64, 0)

In [15]:
# testing:
print(cleanBitcoinData.head())
2 / 0.455

   Unnamed: 0   Timestamp  Open  High   Low  Close  Volume_(BTC)  \
0           0  1325317920  4.39  4.39  4.39   4.39      0.455581   
1         478  1325346600  4.39  4.39  4.39   4.39     48.000000   
2         547  1325350740  4.50  4.57  4.50   4.57     37.862297   
3         548  1325350800  4.58  4.58  4.58   4.58      9.000000   
4        1224  1325391360  4.58  4.58  4.58   4.58      1.502000   

   Volume_(Currency)  Weighted_Price  
0           2.000000        4.390000  
1         210.720000        4.390000  
2         171.380338        4.526411  
3          41.220000        4.580000  
4           6.879160        4.580000  


4.395604395604395